In [ ]:
import nltk
from nltk.corpus import stopwords
import numpy as np
import matplotlib as plt
import pandas as pd

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff


import matplotlib.pyplot as plt

# Новый раздел

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df=pd.read_csv('drive/MyDrive/hw_11/train.csv', encoding = 'unicode_escape')

In [12]:
#df = pd.read_csv("train.csv", encoding = 'unicode_escape')
df['Review_Text']
df.head()

,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0


In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
sw_eng = set(stopwords.words('english'))
len(list(sw_eng))

179

Итак, начнем ваять наше чудо.
Для начала посмотрим, как вообще выглядит один отзыв из нашей таблички - чтобы понять, что убирать и что фиксить

In [16]:
df['Review_Text'][0]

'Excellent room and exercise facility. All around atmosphere was calm and comfortable. Main dining room offers both excellent food and service. Avoid Flying Elephant restaurant, stick to the main dining room.'

In [17]:
y = df['Rating'].unique()
y

array([ 80., 100.,  71.,  86.,  40.,  57.,  43.,  20.,  60.,  65.,  29.])

Попробуем вычленить какие-либо фичи, которые в теории могут помочь улучшить качество нашей модели.
Исходя из богатого 18-летнего опыта (ирония кнш), если человку что-то не понравилось - он это разгромит в пух и прах, не пожалев слов.
Если же наоборот - человеку все понравилось - отзыва либо вообще не будет, либо будет короткий.
Проверим данное на практике - соблюдается ли такое распределение для наших отелей. (добавим фичу - длина отзыва)

In [18]:
df['Len'] = df.apply(lambda x: len(x['Review_Text']), axis=1)

In [19]:
df_for_graph = df[['Len', 'Rating']]

In [20]:
df[['Len', 'Rating']].sort_values(by = 'Rating', ascending = False).head(20)

,Len,Rating
1065,281,100.0
1957,198,100.0
1887,219,100.0
1888,226,100.0
1890,274,100.0
400,115,100.0
399,273,100.0
1394,255,100.0
1894,279,100.0
1896,56,100.0


In [21]:
df[['Len', 'Rating']].sort_values(by = 'Rating', ascending = False).tail(20)

,Len,Rating
1568,272,20.0
2200,90,20.0
136,44,20.0
833,54,20.0
1340,211,20.0
543,449,20.0
2140,68,20.0
1712,385,20.0
175,147,20.0
1005,206,20.0


Чет странно - судя даже по 20 с начала и 20 с конца, баланс, основанный на богатом 18-летнем опыте (снова ирония), нифига не соблюдается.
Чтобы полностью разрушить наши детские мечты, посмотрим на график точек - зависимость рейтинга от длины отзыва

In [28]:
fig = px.scatter(df_for_graph, x = 'Len', y = 'Rating')
fig.show(render = 'colab')

Сверху может не отображаться график - у меня не работала лемматизация в Юпитере и я ушел в колаб переделывать - простите, если графика не будет

Да. к сожалению, фича оказалась полностью бесполезной - люди фактически вне зависимости от рейтинга отеля могут оставить как длинный, так и короткий отзыв. Это идет от того. что на множество отелей писал отзывы не один и тот же человек, а множество разных - поэтому фича (говно) нерабочая -(

Автор подумает над еще новыми фичами, а пока перейдем к другой теме.

Что мы можем сделать на этапе предобработки
Самое начальное:
   1. Убрать все то, что не буква и не слово и не английское
   2. Перевести все в нижний регистр
   3. Убрать стоп-слова из nltk stopwords
   
План есть, работаем

In [29]:
df

,Id,Hotel_name,Review_Title,Review_Text,Rating,Len
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,207
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,171
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,254
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,225
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,250
...,...,...,...,...,...,...
2346,2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,354
2347,2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0,381
2348,2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0,212
2349,2349,The Residency Chennai,The Residency Good Centrally located Hotel,The overall experience was good. However the w...,80.0,116


In [30]:
import re
def clean_text(text):
    text = re.sub(r'[^A-z]',r' ', str(text)).lower() #удалим все, что не английское и переведем в нижний регистр
    clean_text = ' '.join([word for word in text.split() if not word in sw_eng])
    return clean_text

df_1 = df.copy(deep = True)
df_1['Clean Review Title'] = df_1['Review'].apply(clean_text)
df_1['Clean Review Text'] = df_1['Review_Text'].apply(clean_text)

In [31]:
df_1

,Id,Hotel_name,Review_Title,Review_Text,Rating,Len,Clean Review Title,Clean Review Text
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,207,refuge chennai,excellent room exercise facility around atmosp...
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,171,hilton chennai,comfortable felt safe staff helpful respectful...
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,254,worth rating shown websites pricing ok,worth rating shown service good room well main...
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,225,good stay,first nice courteous staff one con stay time c...
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,250,needs improvement,overall ambience hotel good room facilities ne...
...,...,...,...,...,...,...,...,...
2346,2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,354,nan,impressive service staff areas good restaurant...
2347,2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0,381,homely villa,new woodlands chennai gave homely experience l...
2348,2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0,212,nice accommodation facilities,awesome liked neatness maintenance facilities ...
2349,2349,The Residency Chennai,The Residency Good Centrally located Hotel,The overall experience was good. However the w...,80.0,116,residency good centrally located hotel,overall experience good however wi fi getting ...


Приступим ко второй части Марлезонского балета - лемматизации.
Надеюсь, хотя бы тут она у меня заработает

In [38]:
from nltk.corpus import stopwords
from nltk import wordnet, pos_tag, WordNetLemmatizer, ngrams 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [39]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN

def my_lemmatizer(sent):
  if sent!='nan':
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])  
  else:
    return ''

In [40]:
df_1['Title lemmatizer'] = df_1['Clean Review Title'].apply(my_lemmatizer)
df_1['Text lemmatizer'] = df_1['Clean Review Text'].apply(my_lemmatizer)

In [42]:
df_1['Text lemmatizer'][0] #о, а тут работает - живем!

'excellent room exercise facility around atmosphere calm comfortable main dining room offer excellent food service avoid fly elephant restaurant stick main dining room'

Вспомним, что в репозиторий нам также накидали такое чудо, как 2 текстовых файла, содержащих слова из плохих отзывов и из хороших 
Откроем два файла и посмотрим каждый отзыв на наличие таких слов

In [44]:
f = open('drive/MyDrive/hw_11/positive-words.txt', 'r')   
positive = f.read().split("\n")
f.close()

f = open('drive/MyDrive/hw_11/negative-words.txt', 'r', encoding= 'unicode_escape')   
negative = f.read().split("\n")
f.close()

Добавим столбцы с поизитвными и отрицательными словами в тексте сообщения - не думаю, что титульник будет играть большую роль

In [45]:
def positive_count(text):  
  return [word for word in text.split() if word in positive]

def negative_count(text):   
  return [word for word in text.split() if word in negative]

df_1['Positive count'] = df_1['Text lemmatizer'].apply(positive_count)
df_1['Negative count'] = df_1['Text lemmatizer'].apply(negative_count)

In [46]:
df_1

,Id,Hotel_name,Review_Title,Review_Text,Rating,Len,Clean Review Title,Clean Review Text,Title lemmatizer,Text lemmatizer,Positive count,Negative count
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,207,refuge chennai,excellent room exercise facility around atmosp...,refuge chennai,excellent room exercise facility around atmosp...,"[excellent, calm, comfortable, excellent]",[]
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,171,hilton chennai,comfortable felt safe staff helpful respectful...,hilton chennai,comfortable felt safe staff helpful respectful...,"[comfortable, safe, helpful, respectful]",[]
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,254,worth rating shown websites pricing ok,worth rating shown service good room well main...,worth rating show website price ok,worth rating show service good room well maint...,"[worth, good, well, spacious, work, good, wise...",[poor]
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,225,good stay,first nice courteous staff one con stay time c...,good stay,first nice courteous staff one con stay time c...,"[nice, courteous, neat, exceptional]","[inappropriate, disappointed]"
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,250,needs improvement,overall ambience hotel good room facilities ne...,need improvement,overall ambience hotel good room facility need...,"[good, improve]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
2346,2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,354,nan,impressive service staff areas good restaurant...,,impressive service staff area good restaurant ...,"[impressive, good, beautiful, love, creative, ...",[]
2347,2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0,381,homely villa,new woodlands chennai gave homely experience l...,homely villa,new woodland chennai give homely experience lu...,"[luxurious, pretty, good, delicious, lively, t...",[]
2348,2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0,212,nice accommodation facilities,awesome liked neatness maintenance facilities ...,nice accommodation facility,awesome like neatness maintenance facility rea...,"[awesome, like, reasonable, good, happy, comfo...",[]
2349,2349,The Residency Chennai,The Residency Good Centrally located Hotel,The overall experience was good. However the w...,80.0,116,residency good centrally located hotel,overall experience good however wi fi getting ...,residency good centrally locate hotel,overall experience good however wi fi get disc...,[good],[expensive]


Слегка переделаем столбец с рейтингами и уже перейдем к делению выборки 

In [49]:
df_1['Rating'] = df_1['Rating'].apply(lambda x: int(x))
df_1.head(3)

,Id,Hotel_name,Review_Title,Review_Text,Rating,Len,Clean Review Title,Clean Review Text,Title lemmatizer,Text lemmatizer,Positive count,Negative count
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80,207,refuge chennai,excellent room exercise facility around atmosp...,refuge chennai,excellent room exercise facility around atmosp...,"[excellent, calm, comfortable, excellent]",[]
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100,171,hilton chennai,comfortable felt safe staff helpful respectful...,hilton chennai,comfortable felt safe staff helpful respectful...,"[comfortable, safe, helpful, respectful]",[]
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71,254,worth rating shown websites pricing ok,worth rating shown service good room well main...,worth rating show website price ok,worth rating show service good room well maint...,"[worth, good, well, spacious, work, good, wise...",[poor]


Не думаю, что изначально нам пригодится длина отзыва - как мы уже выяснии, она имеет мало общего с оценкой; также выбросим первоначальные версси текстов и оставим только текст после лемматиазции

In [51]:
df_2 = df_1.drop(columns = ['Review', 'Review_Text','Clean Review Title', 'Clean Review Text', 'Len'])
df_2.head(3)

,Id,Hotel_name,Rating,Title lemmatizer,Text lemmatizer,Positive count,Negative count
0,0,Park Hyatt,80,refuge chennai,excellent room exercise facility around atmosp...,"[excellent, calm, comfortable, excellent]",[]
1,1,Hilton Chennai,100,hilton chennai,comfortable felt safe staff helpful respectful...,"[comfortable, safe, helpful, respectful]",[]
2,2,The Royal Regency,71,worth rating show website price ok,worth rating show service good room well maint...,"[worth, good, well, spacious, work, good, wise...",[poor]


In [52]:
X_train, X_test, y_train, y_test = train_test_split(df_2, df_2['Rating'], test_size=0.2, random_state=42)

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train['Text lemmatizer'])
X_test_counts = count_vect.transform(X_test['Text lemmatizer'])

In [86]:
svm = SGDClassifier()
svm.fit(X_train_counts, y_train)
predicted = svm.predict(X_test_counts)

In [88]:
print('Text')
print(metrics.classification_report(y_test, predicted))

Text
              precision    recall  f1-score   support

          20       0.44      0.41      0.42        17
          29       0.25      0.17      0.20        12
          40       0.62      0.33      0.43        24
          43       0.67      0.29      0.40        21
          57       0.26      0.32      0.29        28
          60       0.41      0.52      0.46        31
          65       0.20      0.31      0.24        29
          71       0.51      0.54      0.52        63
          80       0.62      0.51      0.56        61
          86       0.62      0.53      0.57       102
         100       0.63      0.78      0.70        83

    accuracy                           0.51       471
   macro avg       0.47      0.43      0.44       471
weighted avg       0.53      0.51      0.51       471



Учитывая, что титульник - краткое сжатие текстовой информации, не вижу смысла тестить на нем ту же мождель - качество будет в среднем выше на полном тексте

Попробуем обьединить все найденные нами фичи в одну большую фичу

In [92]:
#X_train_big = pd.merge(X_train_counts, X_train[['Negative count','Positive count']], left_on='Id', right_index=True)
#X_test_big = pd.merge(X_test_counts, X_test[['Negative count','Positive count']], left_on='Id', right_index=True)

В общем, как быстро замерджить все признаки в один столбец я чет не могу додуматься, времени маловато осталось - поэтому если я успею придумать, допишу; а пока переходим к обученни моделей

## **Обучение модельки**

In [94]:
tfidf_vect_text = TfidfVectorizer(ngram_range=(1,3))
X_train_tfidf_text = tfidf_vect_text.fit_transform(X_train['Text lemmatizer'])
X_test_tfidf_text = tfidf_vect_text.transform(X_test['Text lemmatizer'])

svm_text = SGDClassifier()
svm_text.fit(X_train_tfidf_text, y_train)
predicted = svm_text.predict(X_test_tfidf_text)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

          20       0.53      0.47      0.50        17
          29       0.50      0.25      0.33        12
          40       0.80      0.33      0.47        24
          43       0.78      0.33      0.47        21
          57       0.69      0.32      0.44        28
          60       0.75      0.39      0.51        31
          65       0.58      0.24      0.34        29
          71       0.41      0.52      0.46        63
          80       0.69      0.56      0.62        61
          86       0.48      0.74      0.58       102
         100       0.64      0.81      0.72        83

    accuracy                           0.56       471
   macro avg       0.62      0.45      0.49       471
weighted avg       0.60      0.56      0.55       471



Результаты по Tf-idf немногим лучше, чем по countVectoirizer - будем использовать ее

In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [100]:
rf_cv = RandomForestClassifier(n_estimators=1000, random_state=0)
rf_cv.fit(X_train_tfidf_text, y_train)
y_pred = rf_cv.predict(X_test_tfidf_text)
acc = accuracy_score(y_test,y_pred),
prec = precision_score(y_test,y_pred, average='macro')
recall =  recall_score(y_test,y_pred, average='macro')
f1 = f1_score(y_test,y_pred, average='macro')

print("accuracy:", acc, "precision:", prec, "recall:", recall, "f1 score:", f1)

accuracy: (0.5031847133757962,) precision: 0.6824012253140175 recall: 0.38629364072949635 f1 score: 0.4197681872420764


In [103]:
log_reg = LogisticRegression()
log_reg.fit(X_train_tfidf_text, y_train)
y_pred = log_reg.predict(X_test_tfidf_text)
acc = accuracy_score(y_test,y_pred),
prec = precision_score(y_test,y_pred, average='macro')
recall =  recall_score(y_test,y_pred, average='macro')
f1 = f1_score(y_test,y_pred, average='macro')

print("accuracy:", acc,  "recall:", recall, "f1 score:", f1)

accuracy: (0.5307855626326964,) recall: 0.3464818774976837 f1 score: 0.3751143178594731


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [108]:
gb = GradientBoostingClassifier()
gb.fit(X_train_tfidf_text, y_train)
y_pred = gb.predict(X_test_tfidf_text)
acc = accuracy_score(y_test,y_pred),
prec = precision_score(y_test,y_pred, average='macro')
recall =  recall_score(y_test,y_pred, average='macro')
f1 = f1_score(y_test,y_pred, average='macro')

print("accuracy:", acc, "precision:", prec, "recall:", recall, "f1 score:", f1)

accuracy: (0.49469214437367304,) precision: 0.4504264882892688 recall: 0.41236500418487004 f1 score: 0.41516018739486965


In [111]:
lin_vec = LinearSVC()
lin_vec.fit(X_train_tfidf_text, y_train)
y_pred = lin_vec.predict(X_test_tfidf_text)
acc = accuracy_score(y_test,y_pred),
prec = precision_score(y_test,y_pred, average='macro')
recall =  recall_score(y_test,y_pred, average='macro')
f1 = f1_score(y_test,y_pred, average='macro')

print("accuracy:", acc, "precision:", prec, "recall:", recall, "f1 score:", f1)

accuracy: (0.5605095541401274,) precision: 0.6046687915302509 recall: 0.4401019331879663 f1 score: 0.47920647713820047


Я не знаю почему, но LinearSVC показало наибольшую точность...ну пусть будет так, ок

## Окончательное сохранение и вывод

In [134]:
tfidf = TfidfVectorizer(ngram_range=(1,3))
X_train_tfidf = tfidf.fit_transform(X_train['Text lemmatizer'])
X_test_tfidf = tfidf.transform(X_test['Text lemmatizer'])

lin_vec = LinearSVC()
lin_vec.fit(X_train_tfidf, y_train)
predicted = lin_vec.predict(X_test_tfidf)
print(metrics.classification_report(y_test, predicted)) 

              precision    recall  f1-score   support

          20       0.53      0.47      0.50        17
          29       0.40      0.17      0.24        12
          40       0.89      0.33      0.48        24
          43       0.67      0.29      0.40        21
          57       0.60      0.32      0.42        28
          60       0.63      0.39      0.48        31
          65       0.64      0.24      0.35        29
          71       0.44      0.48      0.46        63
          80       0.74      0.56      0.64        61
          86       0.48      0.79      0.60       102
         100       0.63      0.81      0.71        83

    accuracy                           0.56       471
   macro avg       0.60      0.44      0.48       471
weighted avg       0.59      0.56      0.54       471



In [135]:
df_test=pd.read_csv('drive/MyDrive/hw_11/test.csv', encoding = 'unicode_escape')
df_test['Clean Review Text'] = df_test['Review_Text'].apply(clean_text)
df_test['Text lemmatizer'] = df_test['Clean Review Text'].apply(my_lemmatizer)

X_test_tfidf = tfidf.transform(df_test['Text lemmatizer'])
predict = lin_vec.predict(X_test_tfidf)

result = pd.DataFrame.from_dict({'Id':df_test.Id.values, 'Rating': lin_vec.predict(X_test_tfidf)})

In [136]:
result

,Id,Rating
0,2351,100
1,2352,100
2,2353,71
3,2354,60
4,2355,86
...,...,...
2348,4698,86
2349,4699,86
2350,4700,71
2351,4701,40


Вывод по заданию:
1. Автор попробовал добавить новые фичи, такие как длина отзыва и набор положительных/отрицательных слов в отзыве, но по причине собственной тупости не смог собрать все это воедино и в итоге выводил результаты на осное только текста
2. Есть идея для нового признака, который (в теории) может улучшить качество предскзааний - так как каждое слово имеет эмоциональный окрас, можно каждому из слов подобрать собственный вес. Также стоит перебрать предложенные слова хороших и плохих слов - например, "expensive" не есть прям совсем недостаток, который категорически везде стоит учитывать
3. Автор не опоздал с дедлайном. Ура.
4. Стоит еще поработать с данными - повыводить зависимости - думаю, там можно много что еще откопать

Общий итог - я недоволен своей крисорукостью, но при этом результат не самый плохой даже на такой грубой и тупой обработке данных -> возможно, используя продвинутые методы, можно добить хотя бы до 0,7 